In [5]:
import os
import glob
import pathlib2 as path
from itertools import count
import json
import re

import h5py as h5
import numpy as np
from skimage.external.tifffile import imread, imsave
from skimage import filters
from skimage import morphology
from scipy import ndimage as ndi
from matplotlib import pyplot as plt
import read_lif

# Export for Classification

In [6]:
rootdir = 'C:/Users/david/Desktop/replication-data/raw/J1_pattern_PCNA594_EdU488_DAPI/'
outdir = 'C:/Users/david/Desktop/replication-data/fixed_single_channel2/'

if not os.path.exists(outdir):
    os.makedirs(outdir)

channel_names = ['pcna', 'edu', 'dapi']

files_staged = glob.glob(rootdir + '*/*.tif')
files_raw = glob.glob(rootdir + '*.lif')

new data -> meta

In [13]:
# Metadata -> simple naming
metadata_file = 'C:/Users/david/Desktop/replication-data/metadata-fixed.json'

with open(metadata_file, 'r') as fd:
    meta = json.load(fd)

p = re.compile('.*_(.*?)chase\.lif.*?')
counter = count()

for c, file in zip(counter, files_staged):
    stage_identification = file.split(os.sep)[-2]
    chase_dur = p.match(file).groups()[0]
    
    name = path.Path(file).name
    original_file = name.split(' - ')[0].split('_', 1)[1]
    series = name.split(' - ')[1].split('-', 1)[0]
    
    out_file = os.path.join(outdir, '_'.join(['metadata', stage_identification, chase_dur, str(c)]) + '.json')
    with open(out_file, 'w') as fd:
        res = meta[original_file][series]
        res['original_file'] = original_file
        res['original_series'] = series
        json.dump(res, fd, indent=1)

new data -> singlechannel tif

In [9]:
p = re.compile('.*_(.*?)chase\.lif.*?')
counter = count()

for c, file in zip(counter, files_staged):
    img = imread(file)
    stage_identification = file.split(os.sep)[-2]
    chase_dur = p.match(file).groups()[0]
    
    for i,channel in enumerate(channel_names):
        imsave(os.path.join(outdir, '_'.join([channel, stage_identification, chase_dur, str(c)]) + '.tif'), np.expand_dims(img[:,i], 1), imagej=True)

old data -> singlechannel tif

In [ ]:
counter = count()

for c, file in zip(counter, files_staged):
    img = imread(file)
    stage_identification = file.split(os.sep)[-2]
    
    for i,channel in enumerate(channel_names):
        imsave(os.path.join(outdir, '_'.join([channel, stage_identification, str(c)]) + '.tif'), np.expand_dims(img[:,i], 1), imagej=True)

for file in files_raw:
    series = read_lif.Reader(file).getSeries()
    for c, ser in zip(counter, series):
        for i,channel in enumerate(channel_names):
            img = ser.getFrame(channel=i)
            imsave(os.path.join(outdir, '_'.join([channel, 'unstaged', str(c)]) + '.tif'), np.expand_dims(img, 1), imagej=True)